In [1]:
import json
import numpy as np
import pandas as pd
import requests
import numpy
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

# Research Question: Find the top 10 players with the highest ratio between completed passes and attempted passes for La Liga

### Import the players dataset

In [2]:

players=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\players.json")
players
#Displaying maximum number
pd.set_option('display.max_columns', 14)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Import the event data for England

In [3]:
events_ita=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\events\\events_italy.json")
pd.set_option('display.max_columns', 15)
events_ita.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959,Pass,3158,1H,2.530536,85,180423957
1,8,Simple pass,[{'id': 1801}],20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959,Pass,3158,1H,3.768418,85,180423958
2,7,Touch,[],8306,"[{'y': 17, 'x': 36}, {'y': 56, 'x': 78}]",2575959,Others on the ball,3158,1H,4.868265,72,180423959
3,1,Ground attacking duel,"[{'id': 504}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 56, 'x': 78}, {'y': 15, 'x': 64}]",2575959,Duel,3158,1H,8.114676,11,180423960
4,1,Ground attacking duel,"[{'id': 503}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 15, 'x': 64}, {'y': 15, 'x': 72}]",2575959,Duel,3158,1H,8.647892,11,180423961


#### As we have to calculate the efficiency of the players based on the passes, we filter out the information related to passes. Thus we target the column 'eventName' and choose only those rows which contains information related to passes. 

In [4]:
events_ita['eventName']=events_ita['eventName'].astype(str)
events_ita=events_ita[(events_ita.eventName.str.contains('Pass'))]
pd.set_option('display.max_columns', 15)
events_ita.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959,Pass,3158,1H,2.530536,85,180423957
1,8,Simple pass,[{'id': 1801}],20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959,Pass,3158,1H,3.768418,85,180423958
5,8,Simple pass,[{'id': 1801}],8306,"[{'y': 15, 'x': 72}, {'y': 25, 'x': 72}]",2575959,Pass,3158,1H,10.376066,85,180423962
10,8,Simple pass,[{'id': 1801}],86366,"[{'y': 82, 'x': 38}, {'y': 88, 'x': 43}]",2575959,Pass,3172,1H,16.240563,85,180423979
14,8,Simple pass,[{'id': 1801}],8306,"[{'y': 15, 'x': 61}, {'y': 15, 'x': 37}]",2575959,Pass,3158,1H,19.153235,85,180423968


#### Splitting the column 'Tags' that is a dictionary

In [5]:
TAG1=[]
for i in range(len(events_ita.tags)):
    try:
        TAG1.append(str(events_ita.tags[i][0]['id']))
    except:
        TAG1.append('Notag')
TAG2=[]
for i in range(len(events_ita.tags)):
    try:
        TAG2.append(str(events_ita.tags[i][1]['id']))
    except:
        TAG2.append('Notag')
TAG3=[]
for i in range(len(events_ita.tags)):
    try:
        TAG3.append(str(events_ita.tags[i][2]['id']))
    except:
        TAG3.append('Notag')
TAG4=[]
for i in range(len(events_ita.tags)):
    try:
        TAG4.append(str(events_ita.tags[i][2]['id']))
    except:
        TAG4.append('Notag')
TAG1=pd.DataFrame({'Tag1':TAG1})
TAG2=pd.DataFrame({'Tag2':TAG2})
TAG3=pd.DataFrame({'Tag3':TAG3})
TAG4=pd.DataFrame({'Tag4':TAG4})
TAG=[TAG1,TAG2,TAG3,TAG4]
TAG=pd.concat(TAG ,axis=1, sort=False)
mix_events_ita=[events_ita,TAG]
events_ita=pd.concat(mix_events_ita,  axis=1,  sort=False).reindex(events_ita.index)
events_ita=events_ita.drop('tags',axis=1)
events_ita.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,8327.0,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959.0,Pass,3158.0,1H,2.530536,85,180423957.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,20438.0,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959.0,Pass,3158.0,1H,3.768418,85,180423958.0,1801,Notag,Notag,Notag
5,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 72}, {'y': 25, 'x': 72}]",2575959.0,Pass,3158.0,1H,10.376066,85,180423962.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,86366.0,"[{'y': 82, 'x': 38}, {'y': 88, 'x': 43}]",2575959.0,Pass,3172.0,1H,16.240563,85,180423979.0,1801,Notag,Notag,Notag
14,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 61}, {'y': 15, 'x': 37}]",2575959.0,Pass,3158.0,1H,19.153235,85,180423968.0,1801,Notag,Notag,Notag


In [6]:
events_ita['eventName'] = events_ita['eventName'].astype(str)
events_ita=events_ita[(events_ita.eventName.str.contains('Pass'))]
events_ita.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,8327.0,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959.0,Pass,3158.0,1H,2.530536,85,180423957.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,20438.0,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959.0,Pass,3158.0,1H,3.768418,85,180423958.0,1801,Notag,Notag,Notag
5,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 72}, {'y': 25, 'x': 72}]",2575959.0,Pass,3158.0,1H,10.376066,85,180423962.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,86366.0,"[{'y': 82, 'x': 38}, {'y': 88, 'x': 43}]",2575959.0,Pass,3172.0,1H,16.240563,85,180423979.0,1801,Notag,Notag,Notag
14,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 61}, {'y': 15, 'x': 37}]",2575959.0,Pass,3158.0,1H,19.153235,85,180423968.0,1801,Notag,Notag,Notag


#### Counting the number of passes with respect to the player id

In [7]:
passes=events_ita
passes=passes.drop('positions',axis=1)
passes.playerId=list(passes.playerId)
PLAYERS=[]
for i in range(len(passes.playerId)):
    PLAYERS.append(passes.playerId.iloc[i])
passes_player_ita=pd.DataFrame(PLAYERS , columns=['PlayerId'])
passes_player_ita['Numberofpasses'] = passes_player_ita.groupby('PlayerId')['PlayerId'].transform('count')
passes_player_ita

,PlayerId,Numberofpasses
0,8327.0,704
1,20438.0,1219
2,8306.0,2043
3,86366.0,1051
4,8306.0,2043
...,...,...
337312,70974.0,247
337313,349102.0,323
337314,413041.0,169
337315,206318.0,899


#### In order to set the threshold of the number of passes, we calculate the first quartile of the number of passes. Then we set the lower limit as mean-1.5$\times$sd. The reason behind this is, while dealing with a set of variables, we could give mean-2$\times$ sd. But the value of twice of sd exceeds the mean. So, we go for mean-1.5$\times$sd. Here we are concerned about the players who have played very limited number of matches or insufficient amount of hours. That is why we set the lower limit as the only threshold.

In [8]:
#calculating Mean
#Calculating Standard deviation
Mean=passes_player_ita.Numberofpasses.mean()
std=passes_player_ita.Numberofpasses.std()
Mean=int(Mean)
std=int(std)
#Calculating Lower bound
lower_bound=Mean-std
ID_passes_threshold_ita=[]
frequency_passes_threshold_ita=[]
frequency_passes=[]
ID_passes=[]
for i in range(len(passes_player_ita.Numberofpasses)):
    frequency_passes.append(float(passes_player_ita.Numberofpasses.iloc[i]))
    ID_passes.append(float(passes_player_ita.PlayerId.iloc[i]))
for i in range(len(frequency_passes)):
    if frequency_passes[i]>lower_bound:
        frequency_passes_threshold_ita.append(int(frequency_passes[i]))
        ID_passes_threshold_ita.append(int(ID_passes[i]))

In [9]:
passes_threshold_ita=pd.DataFrame(ID_passes_threshold_ita, columns=['PlayerId'])
passes_threshold_ita.insert(1, "Numberofpasses",frequency_passes_threshold_ita, True)
passes_threshold_ita.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
passes_threshold_ita

,PlayerId,Numberofpasses
0,8327,704
1,20438,1219
2,8306,2043
3,86366,1051
5,20518,1128
...,...,...
157133,8291,687
160230,3344,724
165435,51145,573
172091,7858,674


In [10]:
events_ita_tag2=events_ita.loc[events_ita['Tag2'] == '1801']
events_ita_tag1=events_ita.loc[events_ita['Tag1']=='1801']
events_ita_tag3=events_ita.loc[events_ita['Tag3']=='1801']
events_ita_tag4=events_ita.loc[events_ita['Tag4']=='1801']
events_ita_tag=[events_ita_tag1,events_ita_tag2,events_ita_tag3,events_ita_tag4]
success_ita=pd.concat(events_ita_tag, axis=0)
success_ita.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,8327.0,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959.0,Pass,3158.0,1H,2.530536,85,180423957.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,20438.0,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959.0,Pass,3158.0,1H,3.768418,85,180423958.0,1801,Notag,Notag,Notag
5,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 72}, {'y': 25, 'x': 72}]",2575959.0,Pass,3158.0,1H,10.376066,85,180423962.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,86366.0,"[{'y': 82, 'x': 38}, {'y': 88, 'x': 43}]",2575959.0,Pass,3172.0,1H,16.240563,85,180423979.0,1801,Notag,Notag,Notag
14,8.0,Simple pass,8306.0,"[{'y': 15, 'x': 61}, {'y': 15, 'x': 37}]",2575959.0,Pass,3158.0,1H,19.153235,85,180423968.0,1801,Notag,Notag,Notag


In [11]:
S=[]
for i in range(len(success_ita.playerId)):
   S.append(success_ita.playerId.iloc[i])
success_ita_new=pd.DataFrame(S , columns=['PlayerId'])
success_ita_new['Numberofpasses'] = success_ita_new.groupby('PlayerId')['PlayerId'].transform('count')
success_ita_new.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
success_ita_new

,PlayerId,Numberofpasses
0,8327.0,288
1,20438.0,626
2,8306.0,1004
3,86366.0,523
5,20518.0,548
...,...,...
118470,424241.0,56
124649,352722.0,8
131244,3548.0,3
133979,286225.0,3


In [16]:
final_passes_ita=pd.merge(success_ita_new, passes_threshold_ita, on='PlayerId')
final_passes_ita=final_passes_ita.rename(columns={'Numberofpasses_x':'AccuratePassesNumber','Numberofpasses_y':'TotalPassesNumber'})
final_passes_ita

,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,8327.0,288,704
1,20438.0,626,1219
2,8306.0,1004,2043
3,86366.0,523,1051
4,20518.0,548,1128
...,...,...,...
243,50069.0,211,786
244,20429.0,312,1027
245,20395.0,227,702
246,40806.0,97,609


In [27]:
def make_col(df, col):
    return pd.concat([df.drop([col], axis=1), df[col].apply(pd.Series)], axis=1)
teams = pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\teams.json").rename(columns={'name':'teamname'})
teams = make_col(teams, "area")
teams = teams.drop(columns='city')
teams = teams.rename(columns={'name':'nation'})
itateams = teams[(teams["alpha3code"]=="ITA") & (teams["type"]=="club")]
itaplayer = players.merge(itateams, left_on="currentTeamId", right_on="wyId")
itaplayer=itaplayer.drop(columns=['birthArea','height','role','currentNationalTeamId','officialName','type','middleName','lastName','passportArea','weight','currentTeamId','firstName','foot'],axis=1)

itaplayer=itaplayer.drop(columns=['alpha3code','alpha2code','id','nation','wyId_y','teamname'])
itaplayer=itaplayer.rename(columns={"wyId_x": "PlayerId"})

final_player_passes_ita=pd.merge(itaplayer, final_passes_ita, on='PlayerId')
final_player_passes_ita


,birthDate,PlayerId,shortName,AccuratePassesNumber,TotalPassesNumber
0,1993-12-19,131184,Ali Adnan,262,639
1,1990-05-04,394,B. Nuytinck,569,1177
2,1994-12-03,265172,A. Bar\u00e1k,501,1112
3,1995-05-07,134508,S. Fofana,456,892
4,1997-08-11,300669,A. Bali\u0107,122,519
...,...,...,...,...,...
232,1992-11-10,21468,M. Perin,313,711
233,1993-01-14,22381,Daniel Bessa,549,837
234,1992-03-02,22883,A. Izzo,461,781
235,1985-05-09,20776,L. Rossettini,513,966


In [23]:
Efficiency_ita=[]
for i in range(len(final_player_passes_ita.AccuratePassesNumber)):
    total=final_player_passes_ita.TotalPassesNumber.iloc[i]
    accurate=final_player_passes_ita.AccuratePassesNumber.iloc[i]
    Ratio=accurate/total
    Efficiency_ita.append(Ratio)
Efficiency_ita=pd.DataFrame(Efficiency_ita)

In [24]:
final_player_passes_ita.insert(4,'Efficiency',Efficiency_ita)
final_player_ratio_ita=final_player_passes_ita
final_player_ratio_ita

,birthDate,PlayerId,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1993-12-19,131184,Ali Adnan,262,0.410016,639
1,1990-05-04,394,B. Nuytinck,569,0.483432,1177
2,1994-12-03,265172,A. Bar\u00e1k,501,0.450540,1112
3,1995-05-07,134508,S. Fofana,456,0.511211,892
4,1997-08-11,300669,A. Bali\u0107,122,0.235067,519
...,...,...,...,...,...,...
232,1992-11-10,21468,M. Perin,313,0.440225,711
233,1993-01-14,22381,Daniel Bessa,549,0.655914,837
234,1992-03-02,22883,A. Izzo,461,0.590269,781
235,1985-05-09,20776,L. Rossettini,513,0.531056,966


In [25]:
final_player_ratio_ita.to_json(r'C:\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw2\final_player_ratio_ita.json')

In [26]:
bestefficiency_ita=final_player_ratio_ita.nlargest(10, ['Efficiency']) 
bestefficiency_ita=bestefficiency_ita.reset_index()
bestefficiency_ita=bestefficiency_ita.drop(columns=['index','PlayerId'])
bestefficiency_ita=bestefficiency_ita.set_value(9, 'shortName', 'V.Lindelof')

bestefficiency_ita

C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,birthDate,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1991-02-01,F. Ghoulam,723,0.890394,812
1,1993-01-12,G. Di Chiara,442,0.827715,534
2,1990-08-26,M. Musacchio,689,0.752183,916
3,1985-01-18,R. Montolivo,470,0.697329,674
4,1991-05-01,B. Salamon,592,0.697291,849
5,1989-05-24,A. Taarabt,498,0.693593,718
6,1988-07-03,T. Heurtaux,360,0.688337,523
7,1986-12-07,L. Memushaj,417,0.683607,610
8,1993-01-14,Daniel Bessa,549,0.655914,837
9,1987-07-17,V.Lindelof,465,0.655853,709


### The above table shows the top 10 most efficient players. We can observe that the highest efficiency is 89.03% in the Serie-A.